## Every Dream trainer

You will need your data prepared first before starting!  Don't waste rental fees if you're not ready to upload your files.  Your files should be captioned before you start with either the caption as the filename or in text files for each image alongside the image files.  See main README.md for more details. Tools are available to automatically caption your files.

[Instructions](https://github.com/victorchall/EveryDream-trainer/blob/main/README.md)

If you can sign up for Runpod here (shameless referral link): [Runpod](https://runpod.io?ref=oko38cd0)

If you are confused by the wall of text, join the discord here: [EveryDream Discord](https://discord.gg/uheqxU6sXN)

Make sure you have at least 40GB of Runpod **Volume** storage at a minimum so you don't waste training just 1 ckpt that is overtrained and have to start over.  Penny pinching on storage is ultimately a waste of your time and money!  This is setup to give you more than one ckpt so you don't overtrain.

### Starting model
Make sure you have your hugging face token ready to download the 1.5 mode. You can get one here: https://huggingface.co/settings/tokens
If you don't have a User Access Token, create one.  Or you can upload a starting checkpoint instead of using the HF download and skip that step, but you'll need to modify the starting model name when you start training (more info below).

In [1]:
# check system resources, make sure your GPU actually has 24GB
# You should see "0 MB / 24576 MB" in the middle of the printout
# if you see 0 MB / 22000 MB find a different instance or provider...
!grep Swap /proc/meminfo
!swapon -s
!nvidia-smi

SwapCached:            0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
import codecs
from google.colab import drive
drive.mount('/content/drive')

!mkdir /content/drive/MyDrive/EveryDreamCkpts
!git clone https://github.com/victorchall/everydream-trainer
%cd everydream-trainer

finish_msg = codecs.encode('QBAR', 'rot_13')

print(finish_msg)

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/MyDrive/EveryDreamCkpts’: File exists
Cloning into 'everydream-trainer'...
remote: Enumerating objects: 1246, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 1246 (delta 92), reused 150 (delta 87), pack-reused 1085
Receiving objects: 100% (1246/1246), 50.31 MiB | 34.34 MiB/s, done.
Resolving deltas: 100% (733/733), done.
/content/everydream-trainer
DONE


## Install dependencies

**This will take a couple minutes.  Wait until it says "DONE" to move on.** 
You can ignore "warnings."

In [3]:
!pip install -q omegaconf
!pip install -q einops
!pip install -q pytorch-lightning==1.6.5
!pip install -q test-tube
!pip install -q transformers==4.19.2
!pip install -q kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -q setuptools==59.5.0
!pip install -q pillow==9.0.1
!pip install -q torchmetrics==0.6.0
!pip install -e .
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
import ipywidgets as widgets
print(finish_msg)

     |████████████████████████████████| 79 kB 7.0 MB/s 
     |████████████████████████████████| 117 kB 59.7 MB/s 
     |████████████████████████████████| 41 kB 469 kB/s 
     |████████████████████████████████| 585 kB 28.2 MB/s 
     |████████████████████████████████| 512 kB 56.7 MB/s 
     |████████████████████████████████| 4.2 MB 37.9 MB/s 
     |████████████████████████████████| 182 kB 48.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.0 MB/s 
     |████████████████████████████████| 551 kB 11.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  Cloning https://github.com/CompVis/taming-transformers.git (to revision master) to ./src/taming-transformers
  Running command git clone -q https://github.com/CompVis/taming-transformers.git /content/everydream-trainer/src/taming-transformers
  Running setup.p

## Now that dependencies are installed, ready to move on!

## Log into huggingface
Run the cell below and paste your token into the prompt.  You can get your token from your [huggingface account page](https://huggingface.co/settings/tokens).

The token will not show on the screen, just press enter after you paste it.

Then run the following cell to download the base checkpoint (may take a minute).

In [4]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
# Default is 1.5 with the new VAE, you can change this to another file on huggingface if you want:
from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="panopstor/EveryDream", #@param 
 filename="sd_v1-5_vae.ckpt", #@param 
 use_auth_token=True
)

#@markdown Copy Model To Gdrive to save time in Following runs and space on your instance
Copy_to_Drive = True #@param {type:"boolean"}

if Copy_to_Drive :
  !cp {downloaded_model_path} /content/drive/MyDrive/EveryDreamCkpts

print(downloaded_model_path) # cache location
print(finish_msg)

In [6]:
#@markdown Copy Model To Gdrive to save time in Following runs and space on your instance

!cp /root/.cache/huggingface/hub/models--panopstor--EveryDream/snapshots/c768f08eebba76b42fcebba817f2643885202c81/sd_v1-5_vae.ckpt /content/drive/MyDrive/EveryDreamCkpts

# Upload training files

Ues the navigation on the left to open **"every-dream-trainer"** folder, then open **"input"** and upload your training files using the **up arrow button** above the file explorer.  

You can upload your images while you wait for the base checkpoint to download above, but do not continue until the checkpoint download is done.

If your captions are in .txt files, upload those in the same folder as the images. 

![upload](https://github.com/victorchall/EveryDream-trainer/blob/main/demo/runpodupload.png?raw=1)

You can check there are files in the folder by running the cell below (optional, just prints first 10).

In [ ]:
!ls -U input | head -10
# at least a few image filenames should show below, if not you uploaded to the wrong folder

## Tweak your YAML
You can adjust the YAML file to change the training parameters.  The v1-fine yamls are in the everydream-trainer/configs/stable-diffusion folder.  By default the "v1-finetune_runpod.yaml" is used here with some sane defaults for one subject with 20-30 images.

You can also upload your own in that folder and use it below.

Instructions are here: [EveryDream README](https://github.com/victorchall/EveryDream-trainer/blob/main/README.md) (hopefully you already read this?)

[Runpod YAML](configs/stable-diffusion/v1-finetune_runpod.yaml) is a good starting point for small datasets (30-50 images) and **is the default in the command below**. It will only keep 2 checkpoints.

If you are running on an A100 on Colab or otherwise, you can adjust the batch size up substantially.  Batch size 16 on A100 40GB as been tested as working.  In Colab, use the file navigation on the left to open the yaml.

If you need help with larger training, join the Discord for advice on making a custom yaml.

In [ ]:
#@markdown change to use a custom yaml, but RUN THIS CELL NO MATTER WHAT to set your yaml path

Training_Script = "runpod" #@param ["runpod"] {allow-input: true}
Custom_Model = False #@param {type:"boolean"}
if Training_Script = "runpod"
 my_yaml = "configs/stable-diffusion/v1-finetune_runpod.yaml"

#elif...
#all the elifs....

#@markdown This can be a link on your Gdrive

if Custom_Model :
downloaded_model_path = "/content/gdrive/MyDrive/EveryDreamCkpts/your.ckpt" #@param {type:"string"}

# downloaded_model_path = "v1-5-pruned-emaonly.ckpt"  # this is the default, but you can change it by uncommenting if you uploaded a file into /everydream-trainer
print(downloaded_model_path) # reminder in case something went wrong with download
print(f"yaml set to: {my_yaml}")

## Run the trainer
Next cell runs training.  This will take a while depending on your number of images, repeats, and max_epochs.  

You can watch for test images in the logs folder.

In [ ]:
# run the trainer, wait until it finishes then SCROLL DOWN to the next cell
Feeling_Lucky = False #@param {type:"boolean"}
Disconnect_after_transfer = False #@param {type:"boolean"}
bye =""
Disconnect_after_transfer="False"

if Feeling_Lucky :
   bye =" --delete"

!python main.py --base {my_yaml} -t  --actual_resume {downloaded_model_path} -n test --data_root input

!python scripts/autoprune_all.py $bye
!cp *.ckpt /content/drive/MyDrive/EveryDreamCkpts

if Disconnect_after_transfer :
  runtime.unassign()

print(finish_msg)

## Prune your checkpoints
This will create 2GB pruned files for all your checkpoints and delete the 11GB files.

If you wish to resume, you may want to remove "--delete" command below and manually delete the 11GB files you dont want to keep.  Typically you only need the last 1 checkpoint in 11gb for resuming.

In [ ]:
# prune the ckpts
!python scripts/autoprune_all.py --delete

## Test
Look in the file drawer on the left for your epoch ckpt names and try them out in the cell below one at a time.  You can save time just downloading the one pruned file that looks the best.  Try each out.

Change the prompt and the ckpt_path below to appropriate values.

In [ ]:
!python scripts/txt2img.py --ckpt_path "epoch=02-step=01000.ckpt" \
--n_samples 2 \
--n_iter 4 \
--prompt "a boy and his dog talking a walk down the sidewalk" \
--scale 6.0 \
--outdir outputs 

In [ ]:
# run to show the images here, or use the file drawer on the left to look at them
import glob
from IPython.display import Image, display
for imageName in glob.glob('outputs/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print(imageName)

## Download your checkpoints

Use the cell below to generate links, right click and save as to download.  If you use Colab, you can skip this and use the Gdrive connect instead.

**If the links don't work, you can double left click the ckpt file in the file drawer on the left, then go to "File" menu then "Download"** or use the Hugging Face upload below.

[EveryDream Discord](https://discord.gg/uheqxU6sXN)

In [ ]:
import glob
from IPython.display import FileLink
for f in glob.glob("*.ckpt"):
    display(FileLink(f))
# right click save as to download the ckpt files

# HuggingFace upload
Use the cell below to upload your checkpoint to your personal HuggingFace account if you want instead of manually downloading. You should already be authorized to Huggingface by token if you used the download/token cells above.

Make sure to fill in the three fields at the top. This will only upload one file at a time, so you will need to run the cell below for each file you want to upload.

* You can get your account name from your [HuggingFace account page](https://huggingface.co/settings/account). Look for your "username" field and paste it below.

* You only need to setup a repository one time.  You can create it here: [Create New HF Dataset](https://huggingface.co/new)  Make sure you write down the repo name you make for future use.  You can reuse it later.

* You need to type the name of the ckpts one at a time in the cell below, find them in the left file drawer of your Runpod/Vast/Colab.

In [ ]:
#list ckpts in root that are ready for download
import glob

for f in glob.glob("*.ckpt"):
    print(f)

In [ ]:
!huggingface-cli lfs-enable-largefiles
# fill in these three fields:
hfusername = "panopstor"
reponame = "EveryDream"
ckpt_name = "epoch=01-step=00030-pruned.ckpt"


target_name = ckpt_name.replace('-','').replace('=','')
import os
os.rename(ckpt_name,target_name)
repo_id=f"{hfusername}/{reponame}"
print(f"uploading to HF: {repo_id}/{ckpt_name}")
print("this make take a while...")

from huggingface_hub import HfApi
api = HfApi()
response = api.upload_file(
    path_or_fileobj=target_name,
    path_in_repo=target_name,
    repo_id=repo_id,
    repo_type=None,
    create_pr=1,
)
print(response)
print(finish_msg)
print("go to your repo and accept the PR this created to see your file")

# Gdrive connect

For Colab only, copies your ckpts to your gdrive.  If the EveryDreamCkpts folder already exists in your gdrive there will be an error, but it should still copy your files. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir /content/drive/MyDrive/EveryDreamCkpts
!cp *.ckpt /content/drive/MyDrive/EveryDreamCkpts